In [ ]:
######Step 1########

!pip install flask-ngrok
!pip install tensorflow==1.14
!pip install keras==2.2.5
!pip install mrcnn
!pip install h5py==2.10.0
!pip install ibm_db
!pip install shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
####Visualize.py changes######
######Step 2 - Copy and paste into mrcnn.visualizer##########

from PIL import Image, ImageDraw

def save_image(image, image_name, boxes, masks, class_ids, scores, class_names, filter_classs_names=None,
               scores_thresh=0.1, save_dir=None, mode=0):
    
    mode_list = [0, 1, 2, 3]
    assert mode in mode_list, "mode's value should in mode_list %s" % str(mode_list)

    useful_mask_indices = []

    N = boxes.shape[0]
    if not N:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    for i in range(N):
        class_id = class_ids[i]
        score = scores[i] if scores is not None else None
        if score is None or score < scores_thresh:
            continue

        label = class_names[class_id]
        if (filter_classs_names is not None) and (label not in filter_classs_names):
            continue

        if not np.any(boxes[i]):
            continue

        useful_mask_indices.append(i)

    if len(useful_mask_indices) == 0:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return

    colors = random_colors(len(useful_mask_indices))

    if mode != 3:
        masked_image = image.astype(np.uint8).copy()
    else:
        masked_image = np.zeros(image.shape).astype(np.uint8)

    if mode != 1:
        for index, value in enumerate(useful_mask_indices):
            masked_image = apply_mask(masked_image, masks[:, :, value], colors[index])

    masked_image = Image.fromarray(masked_image)

    if mode == 3:
        masked_image.save(os.path.join(image_name))
        return

    draw = ImageDraw.Draw(masked_image)
    colors = np.array(colors).astype(int) * 255

    for index, value in enumerate(useful_mask_indices):
        class_id = class_ids[value]
        score = scores[value]
        label = class_names[class_id]

        y1, x1, y2, x2 = boxes[value]
        if mode != 2:
            color = tuple(colors[index])
            draw.rectangle((x1, y1, x2, y2), outline=color)

        draw.text((x1, y1), "%s %f" % (label, score), (255, 255, 255))

    masked_image.save(os.path.join(image_name))

In [ ]:
###Step-3#####

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
from shapely.geometry import Polygon
from skimage.measure import find_contours
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
from mrcnn.visualize import save_image
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

import uuid
import base64
from os import listdir
from os.path import isfile, join
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify,request,send_from_directory

import ibm_db

from google.colab import drive


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
drive.mount('/content/drive/')
ROOT_DIR = '/content/drive/MyDrive/insuranceApp'

Mounted at /content/drive/


In [ ]:
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

WEIGHTS_PATH = "/content/drive/MyDrive/insuranceApp/logs/object20221106T0743/mask_rcnn_object_0011.h5"

In [ ]:
class CustomConfig(Config):

    NAME = "object"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 2
    STEPS_PER_EPOCH = 50
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "scratch")
        self.add_class("object", 2, "dent")
        
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir + subset)

        annotations1 = json.load(open(os.path.join(dataset_dir + '/' + "via_region_data_json.json"),'r',encoding="utf8",errors='ignore'))
        annotations = list(annotations1.values())

        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
            
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects: ",objects)
            name_dict = {"scratch": 1, "dent": 2}

            num_ids = [name_dict[a] for a in objects]

            print("numids", num_ids)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)
            
    def load_mask(self, image_id):
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):

        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):    
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
CUSTOM_DIR = "/content/drive/MyDrive/insuranceApp/dataset/"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()

objects:  ['scratch', 'scratch', 'scratch']
numids [1, 1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch', 'scratch']
numids [1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch', 'scratch']
numids [1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch', 'dent']
numids [2, 1, 2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:  ['dent']
numids [2]
objects:  ['scratch']
numids [1]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent'

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /content/drive/MyDrive/insuranceApp/logs/object20221106T0743/mask_rcnn_object_0011.h5
Re-starting from epoch 11


In [ ]:
def prediction(uid,filename):
  # print(uid)
  area = 0
  # path_to_new_image = '/content/drive/MyDrive/pythoncode/insuranceApp/dataset/test/test3.jpg'
  path_to_new_image = os.path.join(ROOT_DIR + '/server/public/' + uid +'/'+ filename)
  print(path_to_new_image)
  image1 = mpimg.imread(path_to_new_image)
  print(np.shape(image1.shape))
  if np.shape(image1.shape)[0] < 3:
    image1 = image1.reshape((*image1.shape, 1))

  # Run object detection
  #print(len([image1]))
  results1 = model.detect([image1], verbose=0)


  # Display results
  # ax = get_ax(1)
  r1 = results1[0]

  def save_co_ordinates(image, boxes, masks, class_ids, class_names):
      image = image.split("/")[-1]
      image_data = []

      for i in range(boxes.shape[0]):

          mask = masks[:, :, i]
          padded_mask = np.zeros(
              (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
          padded_mask[1:-1, 1:-1] = mask
          contours = find_contours(padded_mask, 0.5)
          for verts in contours:
              verts = np.fliplr(verts) - 1
              list_co_ordinates = np.moveaxis(verts, 1, 0).tolist()

              region = {"shape_attributes": {"all_points_x": list_co_ordinates[0],
                                            "all_points_y": list_co_ordinates[1]},
                        }
              image_data.append(region)
      data = {"filename": image, "regions": image_data, "count": boxes.shape[0]}
      return data

  data = save_co_ordinates(path_to_new_image, r1['rois'], r1['masks'], r1['class_ids'], ['scratch','dent'])

  for i in range(data['count']):
    x = data['regions'][i]['shape_attributes']['all_points_x']
    y = data['regions'][i]['shape_attributes']['all_points_y']
    pgon = Polygon(zip(x, y))
    area = pgon.area + area
    print(pgon.area)

  print(data['count'])

  ##############################################################

  visualize.save_image(image1, path_to_new_image, r1['rois'], r1['masks'],
    r1['class_ids'],r1['scores'],dataset.class_names,
    filter_classs_names=['scratch', 'dent'],scores_thresh=0.9,mode=0)

  ###########################################################################

  return round(area,2)

In [ ]:
# prediction('2cbd9bd1','2cbd9bd1_1.jpg')

In [ ]:
!ngrok authtoken "2H8eWbGDXW4viF60odCrATxzFyY_6d2KvavAHc1K9JbK6xRGw"

/bin/bash: ngrok: command not found


In [ ]:
dsn_hostname = "ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "hzh82386"
dsn_pwd = "oTey9hQKSeDethjI"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "31321" 
dsn_protocol = "TCPIP"
dsn_security = "SSL"

In [ ]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=31321;PROTOCOL=TCPIP;UID=hzh82386;PWD=oTey9hQKSeDethjI;SECURITY=SSL;


In [ ]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg())

Connected to database:  BLUDB as user:  hzh82386 on host:  ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [ ]:
# sql_stmt = "CREATE TABLE users (Uid varchar(255), Username varchar(255), Password varchar(255), carName varchar(255), regNo varchar(255), regDate varchar(255), claimAmount varchar(255))"
# stmt = ibm_db.prepare(conn, sql_stmt)

# try:
#     resp = ibm_db.execute(stmt)    
#     print(resp)
# except:
#     print(ibm_db.stmt_error)

True


In [ ]:
# sql_stmt = "CREATE TABLE images (Uid varchar(255), Result varchar(255), Amount varchar(255))"
# stmt = ibm_db.prepare(conn, sql_stmt)

# try:
#     resp = ibm_db.execute(stmt)    
#     print(resp)
# except:
#     print(ibm_db.stmt_error)

True


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route('/register', methods = ['POST','GET'])
def signUp():
    username = request.args.get('username')
    carName = request.args.get('carName')
    regNo = request.args.get('regNo')
    regDate = request.args.get('regDate')
    password = request.args.get('password')

    sql_stmt = "INSERT INTO users VALUES(?,?,?,?,?,?,?)"
    stmt = ibm_db.prepare(conn, sql_stmt)
    uid = uuid.uuid4().hex[:8]
    ibm_db.bind_param(stmt, 1, uid)
    ibm_db.bind_param(stmt, 2, username)
    ibm_db.bind_param(stmt, 3, password)
    ibm_db.bind_param(stmt, 4, carName)
    ibm_db.bind_param(stmt, 5, regNo)
    ibm_db.bind_param(stmt, 6, regDate)
    ibm_db.bind_param(stmt, 7, "")

    try:
        resp = ibm_db.execute(stmt)    
        if resp:
            try:
                resp = jsonify({"success": uid})
                resp.headers.add('Access-Control-Allow-Origin', '*')
                return resp
            except:
                resp = jsonify({"success": False})
                resp.headers.add('Access-Control-Allow-Origin', '*')
                return resp
                
    except:
        print(ibm_db.stmt_error)

@app.route('/login', methods = ['POST','GET'])
def signIn():

    username = request.args.get('username')
    password = request.args.get('password')

    sql_stmt = f"SELECT * FROM users WHERE Username='{username}' AND Password='{password}'"
    stmt = ibm_db.prepare(conn, sql_stmt)

    print(stmt)

    try:
        resp = ibm_db.execute(stmt)         
        row = ibm_db.fetch_both(stmt)
        if row == False:
            resp = jsonify({"uid":False})
            resp.headers.add('Access-Control-Allow-Origin', '*')
            return resp           
        else:
            listData = {
              "uid": row['UID'],
              "username": row['USERNAME'],
              "carname": row['CARNAME'],
              "regno": row['REGNO'],
              "regdate": row['REGDATE'],
              "claim": row['CLAIMAMOUNT']
            }
            resp = jsonify(listData)
            resp.headers.add('Access-Control-Allow-Origin', '*')
            return resp
               
    except:
        print(ibm_db.stmt_error)

@app.route('/postImage', methods = ['POST'])
def setImg():
    
    index = 1    
    # print(request.form)
    data = request.form
    vals = list(data.values())
    vals.pop(0)

    analyzed = []
    insertValues = []
    claimAmount = 0
    

    if not os.path.exists(ROOT_DIR + '/server/public/' + data['uid']):
        os.makedirs(ROOT_DIR + '/server/public/' + data['uid'])

    for x in vals:
        fileName = data['uid']+'_'+str(index)

        with open(ROOT_DIR + '/server/public/' + data['uid'] + '/' + fileName + ".jpg", "wb") as f:
            f.write(base64.decodebytes(str.encode(x)))

        index += 1

    onlyfiles = [f for f in listdir(ROOT_DIR + '/server/public/'+data['uid']+'/') if isfile(join(ROOT_DIR + '/server/public/'+data['uid']+'/', f))]

    print(onlyfiles)

    # prediction(data['uid'],onlyfiles[0])
    # prediction('das231564a','das231564a_1.jpg')

    # time.sleep(20)
    

    for x in onlyfiles:
      listData = {
        "filename": str(x),
        "area": "",
        "amount": 0
      }
      # tmpData = {
      #     "filename": str(x),
      #     "area": "",
      #     "amount": 0
      # }
      result = prediction(str(data['uid']),str(x))
      print(x)
      listData['area'] = (result / 1000)
      listData['amount'] = round((listData['area'] * 200),2)
      claimAmount += listData['amount']
      analyzed.append(listData)
      insertValues.append((data['uid'],str(listData['area']),str(listData['amount'])))
    print(analyzed)
    print(insertValues)

    tuple_of_tuples = tuple([tuple(x) for x in insertValues])

    sql_stmt = f"INSERT INTO images VALUES(?,?,?)"
    stmt = ibm_db.prepare(conn, sql_stmt)

    try:
        exe = ibm_db.execute_many(stmt,tuple_of_tuples)    
        print(exe)    
        sql_stmt_2 = f"update users set claimamount='{str(claimAmount)}' where uid='{data['uid']}'"
        res = ibm_db.exec_immediate(conn, sql_stmt_2)
        resp = jsonify(analyzed)
        resp.headers.add('Access-Control-Allow-Origin', '*')
        return resp
    except:
        resp = jsonify({"uid": 'error'})
        resp.headers.add('Access-Control-Allow-Origin', '*')
        return resp
        print(ibm_db.stmt_error)
    


@app.route('/viewImage', methods = ['POST','GET'])
def viewImg():
    uid = request.args['uid']
    filename = request.args['filename']
    # uid = request.args['uid']    
    # print(list(uid))
    return send_from_directory(ROOT_DIR + '/server/public/'+uid+'/', filename)

@app.route('/getImage', methods = ['POST'])
def getImg():
    uid = request.args['uid']
    imageData = []
    fileCount = 0
    onlyfiles = [f for f in listdir(ROOT_DIR + '/server/public/'+uid+'/') if isfile(join(ROOT_DIR + '/server/public/'+uid+'/', f))]
    print(onlyfiles)
    sql_stmt = f"SELECT * FROM images WHERE uid='{uid}'"
    stmt = ibm_db.prepare(conn, sql_stmt)

    print(stmt)

    try:
        exe = ibm_db.execute(stmt)    
        print(exe)
        row = ibm_db.fetch_both(stmt)
        while(row):
            listData = {
                "filename": str(onlyfiles[fileCount]),
                "area": row[1],
                "amount": row[2]
            }
            imageData.append(listData)
            fileCount += 1
            # print(row[0])
            # print()
            # print()
            row = ibm_db.fetch_both(stmt)
        print(imageData)
        resp = jsonify(imageData)
        resp.headers.add('Access-Control-Allow-Origin', '*')
        return resp
        
    except:
        print(ibm_db.stmt_error)
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d8a4-34-125-183-117.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:02:55] "POST /login?username=sss&password=1111 HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:02:57] "POST /getImage?uid=2cbd9bd1 HTTP/1.1" 200 -


['2cbd9bd1_1.jpg', '2cbd9bd1_2.jpg', '2cbd9bd1_3.jpg', '2cbd9bd1_4.jpg']
True
[{'filename': '2cbd9bd1_1.jpg', 'area': '751.7575', 'amount': '150351.5'}, {'filename': '2cbd9bd1_2.jpg', 'area': '62.0565', 'amount': '12411.3'}, {'filename': '2cbd9bd1_3.jpg', 'area': '31.2505', 'amount': '6250.1'}, {'filename': '2cbd9bd1_4.jpg', 'area': '1.1885', 'amount': '237.7'}]


INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:03:01] "GET /viewImage?uid=2cbd9bd1&filename=2cbd9bd1_1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:03:01] "GET /viewImage?uid=2cbd9bd1&filename=2cbd9bd1_4.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:03:01] "GET /viewImage?uid=2cbd9bd1&filename=2cbd9bd1_2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:03:01] "GET /viewImage?uid=2cbd9bd1&filename=2cbd9bd1_3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 09:47:56] "POST /getImage?uid= HTTP/1.1" 200 -


[]
True
[]


In [ ]:
ibm_db.close(conn)

True